# Equal-Weight S&P 500 Index Fund

### Library imports

In [34]:
import os, numpy as np, pandas as pd, requests, xlsxwriter, math

In [35]:
stocks = pd.read_csv("data/sp_500_stocks.csv")
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [36]:
# Acquiring an API token - make sure to restart kernel 
from secrets import IEX_CLOUD_API_TOKEN

In [42]:
# Making our first API call
# Going to get the following information fromthe API
# - Market capitalisation for each stock
# - price of each stock

symbol = "AAPL"
api_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}" 

data = requests.get(api_url).json()
data

{'avgTotalVolume': 89963993,
 'calculationPrice': 'close',
 'change': 11.44,
 'changePercent': 0.07736,
 'close': 155.75,
 'closeSource': 'faifcloi',
 'closeTime': 1698263432652,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': 160.69,
 'delayedPriceTime': 1725168298124,
 'extendedChange': 0.11,
 'extendedChangePercent': 0.00073,
 'extendedPrice': 161.06,
 'extendedPriceTime': 1682665292031,
 'high': 157.5,
 'highSource': 'mr1a i5uidetedelcey pn ',
 'highTime': 1706308122708,
 'iexAskPrice': None,
 'iexAskSize': None,
 'iexBidPrice': None,
 'iexBidSize': None,
 'iexClose': 157.66,
 'iexCloseTime': 1677972584292,
 'iexLastUpdated': None,
 'iexMarketPercent': None,
 'iexOpen': 154.01,
 'iexOpenTime': 1710215143052,
 'iexRealtimePrice': None,
 'iexRealtimeSize': None,
 'iexVolume': None,
 'lastTradeTime': 1735809851807,
 'latestPrice': 162.48,
 'latestSource': 'Close',
 'latestTime': 'October 28, 2022',
 'latestUpdate': 1733126301966,
 'latestVolume': 166219842,
 'low': 1

In [43]:
# Parsing our API call
price = data['latestPrice']
market_cap = data["marketCap"]

In [44]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [45]:
final_dataframe.append(
    pd.Series([
        symbol, 
        price,
        market_cap,
        'N/A'],
    index = my_columns), 
    ignore_index = True)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,162.48,2509366145945,N/A


In [61]:
# Now we look thorugh every ticker and append the data to our dataframe
# filter out the stockls that have been delisted
stocks = stocks[~stocks['Ticker'].isin(['DISCA', 'HFC','VIAC','WLTW'])]
final_dataframe = pd.DataFrame(columns = my_columns)
for stock in stocks['Ticker']:
    api_url = f"https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}" 
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
    pd.Series([
        stock, 
        data['latestPrice'],
        data['marketCap'],
        'N/A'
    ],
    index = my_columns),
    ignore_index = True)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,141.38,41324484235,N/A
1,AAL,14.00,9264322424,N/A
2,AAP,200.14,11640347641,N/A
3,AAPL,160.78,2534023909532,N/A
4,ABBV,149.54,271564748947,N/A
...,...,...,...,...
496,YUM,123.77,34456489752,N/A
497,ZBH,116.55,24051609397,N/A
498,ZBRA,293.00,15320888117,N/A
499,ZION,51.96,7758826904,N/A


### Using Batch API calls to imporve performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [33]:
 def chunks(lst, n):
        """Yield  successive n-sized cheunks from lst"""
        for i in range(0, len(lst), n):
            yield lst[i:i + n]

In [63]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'], 
                                                   data[symbol]['quote']['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,142.43,41762762453,N/A
1,AAL,14.00,9356508105,N/A
2,AAP,195.19,11568016877,N/A
3,AAPL,155.91,2617143625171,N/A
4,ABBV,151.31,265242439144,N/A
...,...,...,...,...
496,YUM,121.81,33705854897,N/A
497,ZBH,115.83,24715281948,N/A
498,ZBRA,294.00,15315917304,N/A
499,ZION,51.73,8084542361,N/A


In [64]:
# Calculating the numbert of shares to buy
portfolio_size = input("Enter the value of your portfolio: ")

try:
    val = float(portfolio_size)
    print(val)

except ValueError:
    print("That's not a number /nPlease enter a number")
    portfolio_size = input("Enter the value of your portfolio: ")

Enter the value of your portfolio: 100000
100000.0


In [65]:
position_size = val/len(final_dataframe.index)

for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, "Number of shares to Buy"] = math.floor(position_size/final_dataframe['Stock Price'][i])

## Formatting Our Excel Output
XlsxWriter is an excellent package and offers tons of customization. 

In [66]:
# Initliase xlsx writer
writer = pd.ExcelWriter("recommended trades.xlsx", engine = "xlsxwriter")
final_dataframe.to_excel(writer, "Recommended Trades", index = False)

In [67]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [69]:
# writer.sheets["Recommended Trades"].set_column("A:A", 18, string_format)
# writer.sheets["Recommended Trades"].set_column("B:B", 18, string_format)
# writer.sheets["Recommended Trades"].set_column("C:C", 18, string_format)
# writer.sheets["Recommended Trades"].set_column("D:D", 18, string_format)
# writer.save()

writer.sheets['Recommended Trades'].write('A1', "Ticker", string_format)
writer.sheets['Recommended Trades'].write('B1', "Stock Price", dollar_format)
writer.sheets['Recommended Trades'].write('C1', "Market Capitalization", dollar_format)
writer.sheets['Recommended Trades'].write('D1', "Number of Shares to Buy", integer_format)

0

In [70]:
column_formats = {
    'A':['Ticker', string_format],
    'B':['Stock Price', dollar_format],
    'C':['Market Capitalization', dollar_format],
    'D':['Number of Shares to Buy', integer_format]
}


for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

    
writer.save()